# Analysis notebook for Matrix Multiplication benchmark

This notebook generates plots and tables from the results of the matrix multiplication benchmark.  See `benchmarks/matrix_multiplication` for more information on the benchmark itself.

In [ ]:
import dataclasses
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
from plotly import express as ex

In [ ]:
from symforce.benchmarks.matrix_multiplication.generate_matrix_multiplication_benchmark import (
    get_matrices,
)

matrices = [m[0] for m in get_matrices()]

In [ ]:
# This file should be generated by `run_benchmarks.py`
from symforce.benchmarks.run_benchmarks import MatmulBenchmarkConfig

with (Path("../../..") / "benchmark_outputs" / "matrix_multiplication_benchmark_results.pkl").open(
    "rb"
) as f:
    d = pickle.load(f)

# fmt: off
# d = {('b1_ss', 'double', 'sparse', 7, 7, 49, 15): [1110.422429, 14541544066.0, 4528751448.0], ('b1_ss', 'double', 'flattened', 7, 7, 49, 15): [9.130341, 90817350.0, 18226894.0], ('b1_ss', 'double', 'dense_dynamic', 7, 7, 49, 15): [371.027553, 5127401265.0, 1247404369.0], ('b1_ss', 'double', 'dense_fixed', 7, 7, 49, 15): [68.14021, 678885373.0, 419247643.0], ('Tina_DisCog', 'double', 'sparse', 11, 11, 121, 48): [2845.366436, 37224986078.0, 12096487578.0], ('Tina_DisCog', 'double', 'flattened', 11, 11, 121, 48): [33.045554, 470848994.0, 108236516.0], ('Tina_DisCog', 'double', 'dense_dynamic', 11, 11, 121, 48): [496.618231, 6695699808.0, 1805495380.0], ('Tina_DisCog', 'double', 'dense_fixed', 11, 11, 121, 48): [343.571566, 4985289199.0, 1609370025.0], ('n3c4_b2', 'double', 'sparse', 20, 15, 300, 60): [3161.323538, 40365426533.0, 13588919750.0], ('n3c4_b2', 'double', 'flattened', 20, 15, 300, 60): [45.728614, 440863375.0, 92240806.0], ('n3c4_b2', 'double', 'dense_dynamic', 20, 15, 300, 60): [763.965276, 9962125492.0, 2711624958.0], ('n3c4_b2', 'double', 'dense_fixed', 20, 15, 300, 60): [524.794466, 7432520426.0, 2252440577.0], ('bibd_9_3', 'double', 'sparse', 36, 84, 3024, 252): [214.708183, 2409564974.0, 961313801.0], ('bibd_9_3', 'double', 'flattened', 36, 84, 3024, 252): [65.843983, 733343012.0, 311912384.0], ('bibd_9_3', 'double', 'dense_dynamic', 36, 84, 3024, 252): [131.876468, 1579313610.0, 514380660.0], ('bibd_9_3', 'double', 'dense_fixed', 36, 84, 3024, 252): [128.690005, 1482751252.0, 505146003.0], ('lp_sc105', 'double', 'sparse', 105, 163, 17115, 340): [159.657506, 1918261062.0, 739866228.0], ('lp_sc105', 'double', 'flattened', 105, 163, 17115, 340): [40.520398, 460701824.0, 196825221.0], ('lp_sc105', 'double', 'dense_dynamic', 105, 163, 17115, 340): [2074.208572, 18162980911.0, 6383131613.0], ('lp_sc105', 'double', 'dense_fixed', 105, 163, 17115, 340): None}

In [ ]:
def make_dataframe(d, worst_cpu=None):
    # Filter out tests with no results
    d = {k: v for k, v in d.items() if v is not None}

    # Scale relative to worst result
    if worst_cpu is not None:
        d = {k: [x / worst_cpu[k.matrix_name] for x in v] for k, v in d.items()}

    # Remap names
    remap = {
        "sparse": "Sparse",
        "dense_dynamic": "Dense Dynamic",
        "dense_fixed": "Dense Fixed",
        "flattened": "SymForce",
    }

    d = {dataclasses.replace(k, test=remap[k.test]): v for k, v in d.items()}

    df = pd.DataFrame(
        [dataclasses.astuple(key) + tuple(value) for key, value in d.items()],
        columns=("mat", "scalar", "test", "M", "N", "MN", "nnz", "cpu", "instr", "l1-load"),
    )

    return df

In [ ]:
absolute_df = make_dataframe(d)
absolute_df

In [ ]:
def summary_stats(df):
    tests = df[df.scalar == "double"].test.unique()

    best_cpu = {}
    worst_cpu = {}
    shapes = []
    for matrix in matrices:
        cs = []
        for test in tests:
            try:
                cpu = df.cpu[
                    (df.scalar == "double") & (df.mat == matrix) & (df.test == test)
                ].values[0]
            except IndexError:
                cpu = "N/A"
            cs.append(cpu)
        best_cpu[matrix] = min([c for c in cs if not isinstance(c, str)])
        worst_cpu[matrix] = max([c for c in cs if not isinstance(c, str)])

        M = df.M[(df.mat == matrix)].values[0]
        N = df.N[(df.mat == matrix)].values[0]
        nnz = df.nnz[(df.mat == matrix)].values[0]
        shape = f"{M}x{N}, {round(nnz / (M * N) * 100)}\\%"
        shapes.append(shape)

    return tests, best_cpu, worst_cpu, shapes


tests, best_cpu, worst_cpu, shapes = summary_stats(absolute_df)
df = make_dataframe(d, worst_cpu)
df

In [ ]:
fig = ex.bar(
    df[df.scalar == "double"],
    x="mat",
    y="cpu",
    color="test",
    barmode="group",
    labels={"test": "Method", "mat": "Matrix", "cpu": "Relative CPU %"},
    category_orders={"test": ["Sparse", "Dense Dynamic", "Dense Fixed", "SymForce"]},
    color_discrete_map={
        "Sparse": ex.colors.qualitative.Plotly[0],
        "Dense Dynamic": ex.colors.qualitative.Plotly[3],
        "Dense Fixed": ex.colors.qualitative.Plotly[2],
        "SymForce": ex.colors.qualitative.Plotly[1],
    },
)
fig.update_layout(
    yaxis=dict(tickmode="array", tickvals=[0, 0.5, 1.0], ticktext=["0%", "50%", "100%"]),
    xaxis=dict(
        tickmode="array",
        tickvals=[0, 1, 2, 3, 4, 5],
        ticktext=["(a)", "(b)", "(c)", "(d)", "(e)", "(f)"],
    ),
)
fig.show()

In [ ]:
# Print the latex-formatted results table
print("& " + " & ".join(matrices).replace("_", "\\_") + " \\\\")
print("& " + " & ".join(shapes) + " \\\\ \\hline")
for test in tests:
    s = [test.replace("_", " ").title()]
    for matrix in matrices:
        try:
            cpu = absolute_df.cpu[
                (absolute_df.scalar == "double")
                & (absolute_df.mat == matrix)
                & (absolute_df.test == test)
            ].values[0]
        except IndexError:
            cpu = "N/A"
        s.append(cpu)

    def format(c):
        if c in best_cpu.values():
            c = round(c * 1e6, 1)  # ms to ns
            return f"\\textbf{{{c}}}"
        else:
            if isinstance(c, str):
                return c
            c = round(c * 1e6, 1)  # ms to ns
            return f"{c}"

    s = [format(c) for c in s]
    print(" & ".join(s) + " \\\\")